# Agentic RAG: turbocharge your RAG with query reformulation and self-query! 🚀
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

> This tutorial is advanced. You should have notions from [this other cookbook](advanced_rag) first!

> Reminder: Retrieval-Augmented-Generation (RAG) is “using an LLM to answer a user query, but basing the answer on information retrieved from a knowledge base”. It has many advantages over using a vanilla or fine-tuned LLM: to name a few, it allows to ground the answer on true facts and reduce confabulations, it allows to provide the LLM with domain-specific knowledge, and it allows fine-grained control of access to information from the knowledge base.

But vanilla RAG has limitations, most importantly these two:
- It **performs only one retrieval step**: if the results are bad, the generation in turn will be bad.
- __Semantic similarity is computed with the *user query* as a reference__, which might be suboptimal: for instance, the user query will often be a question and the document containing the true answer will be in affirmative voice, so its similarity score will be downgraded compared to other source documents in the interrogative form, leading to a risk of missing the relevant information.

But we can alleviate these problems by making a **RAG agent: very simply, an agent armed with a retriever tool!**

This agent will: ✅ Formulate the query itself and ✅ Critique to re-retrieve if needed.

So it should naively recover some advanced RAG techniques!
- Instead of directly using the user query as the reference in semantic search, the agent formulates itself a reference sentence that can be closer to the targeted documents, as in [HyDE](https://huggingface.co/papers/2212.10496)
- The agent can the generated snippets and re-retrieve if needed, as in [Self-Query](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/)

Let's build this system. 🛠️

Run the line below to install required dependencies:

In [1]:
!pip install pandas langchain sentence-transformers faiss-cpu "transformers[agents]"

We first load a knowledge base on which we want to perform RAG: this dataset is a compilation of the documentation pages for many `huggingface` packages, stored as markdown.

In [2]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

Now we prepare the knowledge base by processing the dataset and storing it into a vector database to be used by the retriever.

We use [LangChain](https://python.langchain.com/) for its excellent vector database utilities.
For the embedding model, we use [thenlper/gte-small](https://huggingface.co/thenlper/gte-small) since it performed well in our `RAG_evaluation` cookbook.

In [3]:
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from tqdm import tqdm

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed.append(new_doc)

print(
    "Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)"
)
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 2647/2647 [00:34<00:00, 75.69it/s] 


Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)


Now that we have the database ready, let’s build a RAG system that answers user queries based on it!

We want our system to select only from the most relevant sources of information, depending on the query.

👉 Let us build our RAG system as an agent that will be free to choose its query and can iterate on the retrieval results!

In [6]:
import json
from transformers.agents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "text",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "text"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [
                f"===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

Now it’s straightforward to create an agent that leverages this tool!

The agent will need these arguments upon initialization:
- *`tools`*: a list of tools that the agent will be able to call.
- *`llm_engine`*: the LLM that powers the agent.

Our `llm_engine` must be a callable that takes as input a list of [messages](https://huggingface.co/docs/transformers/main/chat_templating) and returns text. It also needs to accept a `stop_sequences` argument that indicates when to stop its generation. For convenience, we directly use the `HfEngine` class provided in the package to get a LLM engine that calls our [Inference API](https://huggingface.co/docs/api-inference/en/index).

And we use [CohereForAI/c4ai-command-r-plus](https://huggingface.co/CohereForAI/c4ai-command-r-plus) as the llm engine because:
- It has a long 128k context, which is helpful for processing long source documents
- It is served for free at all times on HF's Inference API!

In [9]:
from transformers.agents import HfEngine, ReactJsonAgent

llm_engine = HfEngine("CohereForAI/c4ai-command-r-plus")

retriever_tool = RetrieverTool(vectordb)
agent = ReactJsonAgent(
    tools=[retriever_tool], llm_engine=llm_engine, max_iterations=4, verbose=2
)

Since we initialized the agent as a `ReactJsonAgent`, it has been automatically given a default system prompt that tells the LLM engine to process step-by-step and generate tool calls as JSON blobs (you could replace this prompt template with your own as needed).

Then when its `.run()` method is launched, the agent takes care of calling the LLM engine, parsing the tool call JSON blobs and executing these tool calls, all in a loop that ends only when the final answer is provided.

In [10]:
agent_output = agent.run("How can I push a model to the Hub?")

print("Final output:")
print(agent_output)

======== New task ========
How can I push a model to the Hub?
System prompt is as follows:
You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: 'retriever', 'final_answer'
The way you use the tools is by specifying a json blob, ending with '<end_action>'.
Specifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}<end_action>

Make sure to have the $INPUT as a dictionary in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

You should ALWAYS use the f

Final output:
To push a model to the Hub, you can make use of the push_to_hub API provided by Hugging Face. An example usage would be: model.push_to_hub() or trainer.push_to_hub(). Additionally, a function called timm.models.hub.push_to_hf_hub is also available to push models to the Hub.


## Agentic RAG vs. standard RAG

Does the agent setup give a performant RAG system? Well, let's measure it.

We will use [meta-llama/Meta-Llama-3-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct) for evaluation since it's one of the strongest OS models we tested for LLM judge use cases.

In [11]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

Before running the test let's make the agent less verbose.

In [13]:
import logging

agent.logger.setLevel(logging.WARNING)

In [56]:
outputs_agentic_rag = []

for example in tqdm(eval_dataset):
    question = example["question"]

    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
{question}"""
    answer = agent.run(enhanced_question)
    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_agentic_rag.append(results_agentic)

  0%|          | 0/65 [00:00<?, ?it/s]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Calling tool: 'retriever' with arguments: {'query': 'tokenizers-linux-x64-musl binary architecture'}
Calling tool: 'retriever' with arguments: {'query': 'tokenizers-linu

Question: What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Answer: The tokenizers-linux-x64-musl binary is designed for the **x86_64-unknown-linux-musl** architecture.
True answer: x86_64-unknown-linux-musl


Calling tool: 'retriever' with arguments: {'query': 'BLIP-Diffusion purpose'}
Calling tool: 'retriever' with arguments: {'query': 'BLIP-Diffusion role'}
Calling tool: 'retriever' with arguments: {'query': 'BLIP-Diffusion: Pre-trained Subject Representation for Controllable Text-to-Image Generation and Editing'}
Calling tool: 'final_answer' with arguments: {'answer': 'BLIP-Diffusion is a text-to-image generation model that enables zero-shot subject-driven generation and control-guided zero-shot generation. It addresses limitations in existing models, particularly in terms of lengthy fine-tuning and difficulties in preserving subject fidelity. BLIP-Diffusion introduces a new multimodal encoder pre-trained to provide subject representation, allowing for efficient fine-tuning and novel subject-driven generation and editing applications.'}
  3%|▎         | 2/65 [00:43<23:00, 21.92s/it]======== New task ========
Using the information contained in your knowledge base, which you can access wit

Question: What is the purpose of the BLIP-Diffusion model?

Answer: BLIP-Diffusion is a text-to-image generation model that enables zero-shot subject-driven generation and control-guided zero-shot generation. It addresses limitations in existing models, particularly in terms of lengthy fine-tuning and difficulties in preserving subject fidelity. BLIP-Diffusion introduces a new multimodal encoder pre-trained to provide subject representation, allowing for efficient fine-tuning and novel subject-driven generation and editing applications.
True answer: The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.


Calling tool: 'retriever' with arguments: {'query': 'Claim authorship of a paper on the Hugging Face Hub'}
Calling tool: 'retriever' with arguments: {'query': 'How does the Hugging Face Hub match papers to users?'}
Calling tool: 'final_answer' with arguments: {'answer': 'The Hugging Face Hub will attempt to automatically match papers to users based on their email address. Users also have the option to manually control which papers show on their profile by visiting their settings. There, they can choose to hide or show specific papers.'}
  5%|▍         | 3/65 [01:07<23:47, 23.02s/it]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered 

...

Question: What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?

Answer: The name of the open-source library created by Hugging Face to simplify Transformer acceleration is Optimum. It can be found and used at the following links: 
- https://github.com/huggingface/optimum-intel
- https://github.com/huggingface/optimum
- https://huggingface.co/docs/optimum/index
True answer: Optimum


Calling tool: 'retriever' with arguments: {'query': 'Gradio library same height parameter in row'}
Calling tool: 'retriever' with arguments: {'query': 'Gradio equal_height parameter for row elements'}
Calling tool: 'final_answer' with arguments: {'answer': "The parameter used to ensure that elements in a row have the same height in Gradio is 'equal_height'. It is passed to the '.style()' method of 'gr.Row()'."}
 98%|█████████▊| 64/65 [24:23<00:24, 24.59s/it]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should no

Question: What parameter is used to ensure that elements in a row have the same height in Gradio?

Answer: The parameter used to ensure that elements in a row have the same height in Gradio is 'equal_height'. It is passed to the '.style()' method of 'gr.Row()'.
True answer: equal_height


Calling tool: 'retriever' with arguments: {'query': 'Install latest version Optimum OpenVINO'}
Calling tool: 'final_answer' with arguments: {'answer': "The command to install the latest version of Optimum with OpenVINO support is: bash\npip install --upgrade-strategy eager optimum['openvino']\n"}
100%|██████████| 65/65 [24:39<00:00, 22.76s/it]

Question: What is the command to install the latest version of Optimum with OpenVINO support?

Answer: The command to install the latest version of Optimum with OpenVINO support is: bash
pip install --upgrade-strategy eager optimum['openvino']

True answer: pip install --upgrade-strategy eager optimum["openvino"]


In [45]:
from huggingface_hub import InferenceClient

reader_llm = InferenceClient("CohereForAI/c4ai-command-r-plus")

outputs_standard_rag = []

for example in tqdm(eval_dataset):
    question = example["question"]
    context = retriever_tool(question)

    prompt = f"""Given the question and supporting documents below, give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
{question}

{context}
"""
    messages = [{"role": "user", "content": prompt}]
    answer = reader_llm.chat_completion(messages).choices[0].message.content

    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_standard_rag.append(results_agentic)

  2%|▏         | 1/65 [00:01<01:33,  1.47s/it]

Question: What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Answer: The `tokenizers-linux-x64-musl` binary is designed for the x86_64-unknown-linux-musl architecture.
True answer: x86_64-unknown-linux-musl


  3%|▎         | 2/65 [00:02<01:30,  1.44s/it]

Question: What is the purpose of the BLIP-Diffusion model?

Answer: BLIP-Diffusion is a model that enables zero-shot subject-driven generation and control-guided zero-shot image generation.
True answer: The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.


  5%|▍         | 3/65 [00:04<01:41,  1.64s/it]

...

Question: What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?

Answer: The name of the open-source library created by Hugging Face to simplify Transformer acceleration is Optimum.
True answer: Optimum


 98%|█████████▊| 64/65 [02:39<00:02,  2.21s/it]

Question: What parameter is used to ensure that elements in a row have the same height in Gradio?

Answer: `equal_height`
True answer: equal_height


100%|██████████| 65/65 [02:42<00:00,  2.51s/it]

Question: What is the command to install the latest version of Optimum with OpenVINO support?

Answer: The command to install the latest version of Optimum with OpenVINO support is:
```bash
pip install --upgrade-strategy eager optimum["openvino,nncf"]
```
True answer: pip install --upgrade-strategy eager optimum["openvino"]


The evaluation prompt follows some of the best principles shown in [our llm_judge cookbook](llm_judge): it follows a small integer Likert scale, has clear criteria, and a description for each score.

In [68]:
EVALUATION_PROMPT = """You are a fair evaluator language model.

You will be given an instruction, a response to evaluate, a reference answer that gets a score of 3, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 3. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 3}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.
5. Do not score conciseness: a correct answer that covers the question should receive max score, even if it contains additional useless information.

The instruction to evaluate:
{instruction}

Response to evaluate:
{response}

Reference Answer (Score 3):
{reference_answer}

Score Rubrics:
[Is the response complete, accurate, and factual based on the reference answer?]
Score 1: The response is completely incomplete, inaccurate, and/or not factual.
Score 2: The response is somewhat complete, accurate, and/or factual.
Score 3: The response is completely complete, accurate, and/or factual.

Feedback:"""

In [69]:
from huggingface_hub import InferenceClient

evaluation_client = InferenceClient("meta-llama/Meta-Llama-3-70B-Instruct")

In [70]:
import pandas as pd

for type, outputs in [
    ("agentic", outputs_agentic_rag),
    ("standard", outputs_standard_rag),
]:
    for experiment in tqdm(outputs):
        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        messages = [
            {"role": "system", "content": "You are a fair evaluator language model."},
            {"role": "user", "content": eval_prompt},
        ]

        eval_result = evaluation_client.text_generation(
            eval_prompt, max_new_tokens=1000
        )
        try:
            feedback, score = [item.strip() for item in eval_result.split("[RESULT]")]
            experiment["eval_score_LLM_judge"] = score
            experiment["eval_feedback_LLM_judge"] = feedback
        except:
            print(f"Parsing failed - output was: {eval_result}")

    results = pd.DataFrame.from_dict(outputs)
    results = results.loc[~results["generated_answer"].str.contains("Error")]
    results["eval_score_LLM_judge_int"] = (
        results["eval_score_LLM_judge"].fillna(1).apply(lambda x: int(x))
    )
    results["eval_score_LLM_judge_int"] = (results["eval_score_LLM_judge_int"] - 1) / 2

    print(
        f"Average score for {type} RAG: {results['eval_score_LLM_judge_int'].mean()*100:.1f}%"
    )

100%|██████████| 65/65 [02:24<00:00,  2.23s/it]


Average score for agentic RAG: 78.5%


100%|██████████| 65/65 [02:17<00:00,  2.12s/it]

Average score for standard RAG: 70.0%


**Let us recap: the Agent setup improves scores by 8.5% compared to a standard RAG!**

This is a great improvement, with a very simple setup 🚀

(For a baseline, using Llama-3-70B without the knowledge base got 36%)